In [1]:
#!pip install -U transformers
#!pip install -q gradio

In [2]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import gradio as gr

In [3]:
# Load GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Set the pad_token to eos_token (end-of-sequence)
tokenizer.pad_token = tokenizer.eos_token

# Define a function to generate a recipe based on ingredients
def generate_recipe(ingredients):
    prompt = f"I have the following ingredients: {ingredients}. What recipe can I make?"

    # Tokenize the input with padding and create an attention mask
    inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True)
    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask  # Attention mask to ignore padding

    # Generate the recipe with the attention mask and pad_token_id properly set
    output_ids = model.generate(
        input_ids, 
        attention_mask=attention_mask,  # Pass attention mask
        max_length=200, 
        num_beams=5, 
        no_repeat_ngram_size=2, 
        early_stopping=True, 
        pad_token_id=tokenizer.pad_token_id  # Properly set the pad_token_id
    )
    
    # Decode the output
    recipe = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    
    return recipe

# Create the Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# Text-to-Recipe Converter")
    gr.Markdown("Enter a list of ingredients and the model will generate a recipe for you.")
    
    ingredients_input = gr.Textbox(label="Enter Ingredients (comma separated)")
    recipe_output = gr.Textbox(label="Generated Recipe")
    
    generate_button = gr.Button("Generate Recipe")
    
    generate_button.click(generate_recipe, inputs=ingredients_input, outputs=recipe_output)

# Launch the interface
demo.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
